# Michael Harris
# Web-Scraping President's Day deals from ebay.com
# February 13, 2021

### Using the eBay president's Day deals URL, we'll loop through the first 10 pages of results and write all of the results to a text file. Then, we'll download the text files into a BeautifulSoup object, extract relevant information from each listing, and write the results into a table on our local MySQL instance. ### 

In [ ]:
# Loop through the first 10 pages and return the URLs of non-sponsored items. Save these into an object 'url_list' to later
# be called to write all URLs to a text file. 
# Note that I tried to find any sponsored items on this page while using incognito and couldn't find any, so I extracted all 
# URLs.

url_list = []

for i in range(1,10):
    url6 = "https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?_pgn="+str(i)
    header = ({'User-Agent':'Mozilla/5.0'}) 
    response = requests.get(url6, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.findAll('div',{"class":"s-item__info clearfix"})
    count = 0
    for item in items:
        a = item.find("a")
        if not a:
            continue
        text = item.find("span").get_text
        url_simplify = re.sub("(.*)\\?.*", r"\1", a['href']) 
        url_list.append(url_simplify + "\n")
        print(url_simplify)
                


# Print URLs to confirm that we loaded them correctly into text file. 

In [ ]:
# Next, we'll open the url_list object that we created, and for each line, write down the URL and then indent to another line.
# Save in 'deals.txt' file.
with open ('deals.txt', 'w') as file:
                for line in url_list:
                    file.write(line + "\n")
                    file.close

In [ ]:
# Open 'deals.txt' and save each URL into a html file, naming each file file the item ID of the item that we're scraping using 
# Regular Expressions. To pass through errors, set errors = 'ignore', so that our code doesn't break if a URL isn't valid.

with open('deals.txt', encoding = 'UTF-8',errors="ignore") as inf:
    for url in inf: 
        if 'http' in url:
            page = requests.get(url.strip())
            response = urlopen(url)
            page = response.read()
            text_after = re.sub('(.+?)/(.+?)/(.+?)/(.+?)/(.+)\n', '\g<5>' , url)
            with open(text_after + '.htm', 'wb') as outfile:
                outfile.write(page)
                time.sleep(4)
                
# Adding time.sleep command to avoid overloading servers.

In [ ]:
# Now that we've named the files by their item name, have downloaded the HTML from the webpage for each URL, and have saved
# each file individually in a folder called 'deals', we can write a loop to loop through the the downloaded pages and 
# parse them into a BeautifulSoup object.

# Create list objects to store results of the for loop iterations, that we'll later use to import to MySQL. 
seller_list = []
sellerrating_list = []
item_price_list = []
list_price_list = []
numbersold_list = [] 
condition_list = []



# Loop through html files in the 'deals' directory.
directory ='/Users/MIckey Harris/Downloads/deals'
for filename in os.listdir(directory):
    if filename.endswith('.htm'):
        fname = os.path.join(directory,filename)
        with open(fname, encoding = 'UTF-8', errors = 'ignore') as f:
            soup = BeautifulSoup(f.read(),'html.parser')
            seller = soup.findAll('div', {'class': 'mbg vi-VR-margBtm3'}) # Found all class objects that contained seller name.
            for item in seller:
                b = item.find('span').get_text()                          # Returned text for all sellers.
                seller_list.append(b)
            sellerrating = soup.findAll('span', {'class': 'mbg-l'})       # Found all class objects that contained seller rating.
            for item in sellerrating:
                a = item.find('a').get_text()                             # Returned text for all seller ratings.
                sellerrating_list.append(a)    
            item_price = soup.findAll('div', {'id': 'vi-mskumap-none'})   # Found all div objects w/ ID that contained item price.
            for item in item_price:
                c = item.find('span')['content']                          # Returned text for all item prices.
                item_price_list.append(float(c))
            list_price = soup.findAll('div', {'id':'vi-priceDetails'})    # Found all div objects w/ ID that contained list price.
            for item in list_price:
                h = item.find('span',{'class': 'vi-originalPrice'}).get_text() # Returned text for all list prices.
                list_price_list.append(h)
            numbersold = soup.findAll('a', {'class': 'vi-txt-underline'}) # Found all a objects w/ a certain class that contained number sold.
            for item in numbersold: 
               d = item.get_text()
               d_simplify = re.sub("(.+?)(.+)\n", '\g<1>', d)             # Returned text for all list prices. Use regex to format as number.
               numbersold_list.append(d_simplify)
            condition = soup.findAll('div',{'class':'u-flL condText'})    # Found all div objects w/ a certain class that contained condition.
            for item in condition:
                g = item.get_text()                                       # Returned text for all item conditions.
                condition_list.append(g)
            
            
            

print(seller_list)
print(sellerrating_list)
print(item_price_list)
print(list_price_list)
print(condition_list)
print(numbersold_list)

# We observe that our variables have been stored into list objects as we wanted.

In [ ]:
# Ensure that the lists that we imported are the same length by replacing missing values with 'Null'. 

# First, I'm ensuring that my lists are the same length so that I can import to MySQL. 
# Next, I am replacing missing values with 'NULL'. 

max_length = max(max(len(seller_list), len(sellerrating_list)), len(item_price_list), len(list_price_list), len(condition_list), len(numbersold_list))
seller_list += ['NULL'] * (max_length - len(seller_list))
sellerrating_list += ['NULL'] * (max_length - len(sellerrating_list))
item_price_list += ['NULL'] * (max_length - len(item_price_list))
list_price_list += ['NULL'] * (max_length - len(list_price_list))
condition_list += ['NULL'] * (max_length - len(condition_list))
numbersold_list += ['NULL'] * (max_length - len(numbersold_list))

In [ ]:
# Connect to our MySQL local instance & set up our cursor. Connect to 'msba' database.

db = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "",
    database = 'msba') 

cursor = db.cursor()


In [ ]:
# Create a 'deals' table with SellerName, SellerRating, ItemPrice, ListPrice, NumberSold, and Condition columns. 

cursor.execute("CREATE TABLE IF NOT EXISTS deals (sellername VARCHAR(50), sellerrating VARCHAR(50), itemprice FLOAT, listprice VARCHAR(50), numbersold VARCHAR(50), cond VARCHAR(50))")

cursor.execute("SHOW TABLES")

# Fetch all tables within our database.
tables = cursor.fetchall() 

# Showing all our tables in the database.
for table in tables:
    print(table)
    
# We see that the 'deals' table has been created.

In [ ]:
### PART 3E ###

# List insert query to insert into the 'deals' table our eBay data.
query = "INSERT INTO deals (sellername,sellerrating,itemprice,listprice,numbersold,cond ) VALUES (%s, %s, %s, %s, %s, %s)"


In [ ]:
# Run a for loop to execute insert queries for each of the 5 records that are stored in the 'data' object.

# In this case, I'll simply test that importing my list works by importing the first 5 items in the list.
data = [
    (seller_list[0], sellerrating_list[0],item_price_list[0],list_price_list[0],numbersold_list[0],condition_list[0]),
    (seller_list[1], sellerrating_list[1],item_price_list[1],list_price_list[1],numbersold_list[1],condition_list[1]),
    (seller_list[2], sellerrating_list[2],item_price_list[2],list_price_list[2],numbersold_list[2],condition_list[2]),
    (seller_list[3], sellerrating_list[3],item_price_list[3],list_price_list[3],numbersold_list[3],condition_list[3]),
    (seller_list[4], sellerrating_list[4],item_price_list[4],list_price_list[4],numbersold_list[4],condition_list[4])
]

for values in data:
    cursor.execute(query,values)


# Next, we run the 'commit()' method of the database object to ensure that our insert statement executes. 
db.commit()

# Lastly, print that our records were inserted correctly into the database.
print(cursor.rowcount, "records inserted")


In [ ]:
# Print select query from our database showing that our data was correctly inserted.
query = "SELECT * FROM deals"

# Execute the query.
cursor.execute(query)

# Return all the records in the cursor object.
records = cursor.fetchall()

# Show the data as an output.
for record in records:
    print(record)

# We see that our records have been imported into my MySQL table. 

In [ ]:
# Now, we can write queries from our 'deals' table to determine better purchase decisions. Let's query the average price,
# minumum price, and maximum price.

query = "SELECT AVG(itemprice), min(itemprice), max(itemprice) FROM deals"

# Execute the query.
cursor.execute(query)

# Return all the records in the cursor object.
records = cursor.fetchall()

# Show the data as an output.
for record in records:
    print(record)
    
# Returned are the summary statistics for the ItemPrice column. 

In [ ]:
# As an example, I'll group by sellername, and return summary statistics for the sellers that I imported.
query = "SELECT AVG(itemprice), min(itemprice), max(itemprice) FROM deals GROUP BY sellername"

# Execute the query.
cursor.execute(query)

# Return all the records in the cursor object.
records = cursor.fetchall()

# Show the data as an output.
for record in records:
    print(record)
    
# Returned are the summary statistics for the ItemPrice column. 